In [4]:
from __future__ import print_function

import time
import requests
from bs4 import BeautifulSoup
import pymongo
import pandas as pd
import numpy as np
import re

from sklearn.externals import joblib

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
wnlzer = WordNetLemmatizer()


from pprint import pprint
from time import time
import logging


import sys
import os
import csv
import shutil
from collections import defaultdict
from sklearn.linear_model import SGDClassifier, Perceptron
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV, ParameterGrid
from sklearn.pipeline import Pipeline
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import roc_auc_score
#import parfit.parfit as pf


import matplotlib.pyplot as plt

source_path = 'test_data'
#source_file = 'test_strains_reviews.csv'
target_path = '/Users/jordanweil/green_rex/test_data'

In [8]:
def list_o_strains(i):
    LOS = []
    Type = []
    r = requests.get(i)
    soup2 = BeautifulSoup(r.content, 'html.parser')
    strains = soup2.find_all('a', class_="ga_Explore_Strain_Tile")
    for s in strains:
        LOS.append(str(s.attrs['href'])[1:])
    return LOS

In [327]:
def get_reviews2(l):
    """Pass in a list of URL's and return them in a mongo db table as a dicitonary with 
    {'url', 'html'} and their corresponding values"""
    r = requests.get(l)
    html = (r.content)
    return html

In [463]:
def parse_docs2(d):
    """Parse the HTML docs that we have stored in a dictionary, return as a list.
    Also scrape and parse star rating for each review """
    strain_description = []
    strain_flavors = []
    strain_effects = []
    effects = []
    negs = []
    meds = []
    
    soup = BeautifulSoup(d, 'html.parser')
    desc = soup.find_all('div', itemprop='description') 
    for d in desc:
        text = d.text
        remove_punch = re.sub('[^A-Za-z ]' , "" ,text )
        token = remove_punch.lower().split()
        srm_token = [wnlzer.lemmatize(i) for i in token if not i in set(stopwords.words('english'))]
        clean_text = " ".join(srm_token)
        strain_description.append(clean_text)
    
    #create list of all the effects
    effect = soup.find_all('div', attrs={'class':'histogram-label'})
    for e in effect:
        strain_effects.append(e.text)
        
    # create smaller lists of good, bad, medical effects
    for i in strain_effects[:5]:
        effects.append(i)
    for i in strain_effects[10:]:
        negs.append(i)
    for i in strain_effects[5:10]:
        meds.append(i)
    
    #Create list of each strain flavor
    flavors = soup.find_all('div', class_='flavor-name')
    for f in flavors:
        strain_flavors.append(f.text[3:])
        
    return strain_description, effects, meds, negs, strain_flavors

In [502]:
def strain_dict_entry(strain, desc, flavors, effects, meds, negs, straindict):
    if straindict is None:
        straindict = {}
        
    if strain not in straindict:
        straindict[strain] = {
            "desc" : desc,
            "flavors" : flavors,
            "effect" : []
        }
        
    straindict[strain]['effect'].append({'positive': effects,
                                         'medical': meds,
                                         'negative': negs
                                        })
    return straindict

In [507]:
def scraper_dummy1(los):
    """pass in a list of strains to be scraped from Leafly.
    returns dictionary keyed by strains w/strain info(reviews) as values"""
    cnt = 0
    strain_dict = None
    for s in los:
        strain = s
        url = "https://www.leafly.com/{}".format(strain)
        d = get_reviews2(url)
        desc, effects, meds, negs, flavors = parse_docs2(d)
        if len(desc) == 0:
            break
     
        for description, flavors, effects, meds, negs in zip(desc, flavors, effects, meds, negs):
                strain_dict = strain_dict_entry(strain, description, flavors, effects, meds, negs, strain_dict)
                
        if cnt % 10 == 0:
                print(cnt)
        cnt +=1
            
    return strain_dict      

In [508]:
los = list_o_strains('https://www.leafly.com/explore')

In [ ]:
d = scraper_dummy1(los)

0
10
20
30


In [506]:
d

{'hybrid/blue-dream': {'desc': 'blue dream sativadominant hybrid originating california achieved legendary status among west coast strain crossing blueberry indica sativa haze blue dream balance fullbody relaxation gentle cerebral invigoration novice veteran consumer alike enjoy level effect blue dream ease gently calm euphoria blue dream phenotype express indicalike look feel sativaleaning variety remains prevalentwith sweet berry aroma redolent blueberry parent blue dream delivers swift symptom relief without heavy sedative effect make blue dream popular daytime medicine patient treating pain depression nausea ailment requiring high thc strain',
  'flavors': 'Happy',
  'effect': [{'positive': 'Stress',
    'medical': 'Dry Mouth',
    'negative': 'Blueberry'},
   {'positive': 'Stress', 'medical': 'Dry Mouth', 'negative': 'Blueberry'}]},
 'sativa/sour-diesel': {'desc': 'sour diesel sometimes called sour invigorating sativadominant strainnamed pungent diesellike aroma fastacting strain 